In [80]:
import numpy as np
import cv2
import imutils
import sys
import pytesseract
import pandas as pd
import time

In [81]:
image = cv2.imread('car3.jpeg')

In [82]:
cv2.imshow("Original Image", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [83]:
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
cv2.imshow("1 - Grayscale Conversion", gray)
cv2.waitKey(0)

# Noise removal with iterative bilateral filter(removes noise while preserving edges)
gray = cv2.bilateralFilter(gray, 15, 17, 17)
cv2.imshow("2 - Bilateral Filter", gray)
cv2.waitKey(0)

# Find Edges of the grayscale image
edged = cv2.Canny(gray, 17, 200)
cv2.imshow("4 - Canny Edges", edged)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [84]:
(cnts, _) = cv2.findContours(edged, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
cnts=sorted(cnts, key = cv2.contourArea, reverse = True)[:30] #sort contours based on their area keeping minimum required area as '30' (anything smaller than this will not be considered)
NumberPlateCnt = None #we currently have no Number plate contour


In [85]:
# loop over our contours to find the best possible approximate contour of number plate
count = 0
for c in cnts:
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * peri, True)
        if len(approx) == 4:  # Select the contour with 4 corners
            NumberPlateCnt = approx #This is our approx Number Plate Contour
            break


In [87]:
# Drawing the selected contour on the original image
cv2.drawContours(edged, [NumberPlateCnt], -1, (0,255,0), 3)
cv2.imshow("Final Image With Number Plate Detected", image)

cv2.waitKey(0) #Wait for user input before closing the images displayed
cv2.destroyAllWindows()

In [54]:
# Masking the part other than the number plate
mask = np.zeros(gray.shape,np.uint8)
new_image = cv2.drawContours(mask,[NumberPlateCnt],0,255,-1)
new_image = cv2.bitwise_and(image,image,mask=mask)
cv2.namedWindow("Final_image",cv2.WINDOW_NORMAL)
cv2.imshow("Final_image",new_image)
cv2.waitKey(0) #Wait for user input before closing the images displayed
cv2.destroyAllWindows()

In [22]:
pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract.exe'
# Run tesseract OCR on image
text = pytesseract.image_to_string(new_image,lang='eng')

#Data is stored in CSV file
raw_data = {'date': [time.asctime( time.localtime(time.time()) )],
        'v_number': [text]}

df = pd.DataFrame(raw_data, columns = ['date', 'v_number'])
df.to_csv('data.csv')

# Print recognized text
print(text)

LR33 TEE
